In [1]:
import pandas as pd
import os

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
ORIGIN_DIR = r"../downloads/olist/"
INPUT_DIR = r"../artifact/preprocess/"
TRANSLATED_RESULT_DIR = r"../downloads/backup"
ARTIFACT_DIR = r"../artifact/translated_reviews/"

In [3]:
origin_reviews_path = os.path.join(ORIGIN_DIR, 'olist_order_reviews_dataset.csv')
preprocessed_with_id_path = os.path.join(INPUT_DIR, 'preprocessed_reviews.csv')
preprocessed_path = os.path.join(INPUT_DIR, 'all_portuguess.txt')

p2e_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans-p2e.txt')
e2k_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans-e2k.txt')

os.path.exists(origin_reviews_path)

True

In [4]:
import csv
import pandas as pd

translated_result_df = []
for path in [preprocessed_path, p2e_path, e2k_path]:
    df = pd.read_csv(
        path,
        header=None,
        names=["review"],
        sep=r'\n',
        engine='python',
        quotechar='"',
        doublequote=True
    )
    translated_result_df.append(df)

merged_translation = pd.concat(translated_result_df, axis=1)
merged_translation.columns = ['origin', 'p2e', 'e2k']

# 후처리 필요...
for col in ["origin", "p2e", "e2k"]:
    # 1. 이중 따옴표 먼저 처리
    merged_translation[col] = merged_translation[col].str.replace('""', '"', regex=False)
    # 2. 시작과 끝의 따옴표만 제거 (정규표현식 사용)
    merged_translation[col] = merged_translation[col].str.replace(r'^"(.*)"$', r'\1', regex=True)

merged_translation.head(3)

,origin,p2e,e2k
0,"não recebi o produto, o produto consta como enviado para o correio de ribeirão preto. o correio não recebeu o produto. envie varias mensagem para a targaryen e não obtive. esta targaryen esta sujando seu nome","“I did not receive the product, the product is listed as sent to the post office in Ribeirão Preto. The post office did not receive the product. I sent several messages to Targaryen and did not receive a response. This Targaryen is ruining her name”",“제품을 받지 못했습니다. 제품은 리베이랑 프레투에 있는 우체국으로 보내졌다고 되어 있습니다. 우체국에서는 제품을 받지 못했습니다. 저는 Targaryen에게 메시지를 몇 번 보냈지만 답장을 받지 못했습니다. 이 Targaryen은 자신의 이름을 망치고 있습니다.”
1,"adquiri 2 mochilas pelo site lannister de e parceiros diferentes. a targaryen de mg entregou bem antes do prazo previsto, sendo que a outra da pacific que vem de sc ainda ñ foi entregue. parabéns a targaryen","I bought 2 backpacks from the lannister site and from different partners. targaryen from mg delivered well before the estimated deadline, while the other one from pacific that comes from sc still hasn’t been delivered. congratulations to targaryen",저는 랜니스터 사이트와 다른 파트너로부터 두 개의 배낭을 구매했습니다. 배달 예정일보다 더 일찍 배달된 MG의 타르가리온 배낭과 달리 태양의 권세로부터 온 파시픽의 배낭은 아직 배달되지 않았습니다. 타르가리온에 축하를 보냅니다.
2,"comprei o produto confiando nas lojas lannister e não na targaryen a qual parece q não entregou o produto no tempo certo para os correios, e não recebi esclarecimento da targaryen, mas o produto chegou, obg","I bought the product trusting the lannister stores and not the targaryen one which seems to not have delivered the product in the right time for the post office, and I did not receive any clarification from the targaryen, but the product arrived, thank you","저는 랜니스터 상점을 믿고 제품을 구매했고, 타르가리온 상점은 제품을 적절한 시간에 배송하지 않는 것 같았습니다. 배송 업체에서 제품을 받지 못했습니다. 타르가리온 측에서 아무런 설명도 듣지 못했습니다. 그러나 제품은 도착했습니다. 감사합니다."


In [5]:
preprocessed_with_id_df = pd.read_csv(preprocessed_with_id_path)

# 딕셔너리로 변환
for target in ['p2e', 'e2k']:
    trans_dict = dict(zip(merged_translation["origin"], merged_translation[target]))

    preprocessed_with_id_df[f"title_{target}"]   = preprocessed_with_id_df["review_comment_title"].map(trans_dict)
    preprocessed_with_id_df[f"message_{target}"] = preprocessed_with_id_df["review_comment_message"].map(trans_dict)

preprocessed_with_id_df.shape

(42590, 7)

In [6]:
# !pip install missingno

In [7]:
# import missingno as msno
# msno.matrix(preprocessed_with_id_df)

In [8]:
preprocessed_with_id_df.isnull().sum()

review_id                     0
review_comment_title      31094
review_comment_message     1734
title_p2e                 31094
message_p2e                1735
title_e2k                 31094
message_e2k                1735
dtype: int64

In [9]:
# 원본 title 값이 있는데 번역에서 값이 없는 경우
mask = (
    ~preprocessed_with_id_df['review_comment_title'].isna() &
    preprocessed_with_id_df['title_p2e'].isna()
)
preprocessed_with_id_df[mask][['review_id', 'review_comment_title', 'title_p2e']]

,review_id,review_comment_title,title_p2e


In [10]:
# 원본 review_comment_message 값이 있는데 번역에서 값이 없는 경우
mask = (
    ~preprocessed_with_id_df['review_comment_message'].isna() &
    preprocessed_with_id_df['message_p2e'].isna()
)
preprocessed_with_id_df[mask][['review_id', 'review_comment_message', 'message_p2e']]

,review_id,review_comment_message,message_p2e
21991,e9a917ef5451a4ece4f44bd595ef245b,n/d,NaN


In [11]:
# 원본을 보니 별 내용이 없음.
origin_reviews = pd.read_csv(origin_reviews_path)
origin_reviews[origin_reviews['review_id'] == 'e9a917ef5451a4ece4f44bd595ef245b']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
51119,e9a917ef5451a4ece4f44bd595ef245b,b4700fc91bb3fbc155bfc04ae821ba04,4,NaN,N/D,2017-02-23 00:00:00,2017-02-23 23:17:26


In [12]:
# 다른 order_id에 동일한 리뷰를 작성할 수도 있음
duplicated_rows = preprocessed_with_id_df[preprocessed_with_id_df.duplicated(keep=False)]
# duplicated_rows.sort_values('review_id').head(2)
origin_reviews[origin_reviews['review_id'] == "00130cbe1f9d422698c812ed8ded1919"]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecido pela impressora, consequentemente não funcionou. Além de ter chegado com atraso de mais de 15 dias do previsto. Preciso que seja trocado.",2018-03-07 00:00:00,2018-03-20 18:08:23
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecido pela impressora, consequentemente não funcionou. Além de ter chegado com atraso de mais de 15 dias do previsto. Preciso que seja trocado.",2018-03-07 00:00:00,2018-03-20 18:08:23


In [13]:
# 중복 제거
preprocessed_with_id_df = preprocessed_with_id_df.drop_duplicates()
preprocessed_with_id_df.shape

(42273, 7)

In [14]:
print(len(origin_reviews))
origin_reviews = origin_reviews.drop(columns = ['review_comment_title', 'review_comment_message'])

os.makedirs(ARTIFACT_DIR, exist_ok=True)
result = []
for target in ['p2e', 'e2k']:
    output_df = pd.merge(origin_reviews, preprocessed_with_id_df[['review_id', f'title_{target}', f'message_{target}']], on='review_id', how='left')

    save_path = os.path.join(ARTIFACT_DIR, f'reviews_{target}.csv')
    output_df.to_csv(save_path, index=False)

    result.append(output_df)

    print(len(output_df))

99224
99224
99224
